In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload
import matplotlib.pyplot as plt

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import model_params as params
import utils.resnet_orig as resnet
import utils.gmm_helpers as gmm_helpers

from tensorboardX import SummaryWriter
%load_ext line_profiler

In [2]:
def show_stats(model, device, model_params):
    print(tt.test(model, device, model_params.test_loader, min_conf=.101)[0])
    df = ev.evaluate(model, device, model_params.data_name, model_params.loaders)
    return df

def evaluate(model_params, device, file_joint, file_base, file_gmm):
    model_joint = torch.load('SavedModels/' + file_joint).to(device)

    df_joint = show_stats(model_joint, device, model_params)
    df_joint_base = show_stats(model_joint.base_model, device, model_params)

    base_model = torch.load('SavedModels/base/' + file_base)
    gmm = torch.load('SavedModels/GMM/' + file_gmm)

    lam = model_joint.loglam
    model = models.RobustModel(base_model, gmm, lam, dim=model_params.dim).to(device)

    df_sep = show_stats(model, device, model_params)
    df_sep_base = show_stats(model.base_model, device, model_params)
 
    df1 = pd.concat([df_sep_base, df_sep], axis=1, keys=['Base', 'GMM'])
    df2 = pd.concat([df_joint_base, df_joint], axis=1, keys=['Base', 'GMM'])
    return df1, df2 

In [3]:
device = torch.device('cuda:3')
dataset = 'MNIST'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCAMNIST_lam0.0_n100_lr0.001_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_MNIST_lr0.001_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_MNIST_n100_data_used60000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

0.9951
0.9952


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.9948
0.9949
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &  FPR@95 &       MMC &     AUROC &  FPR@95 \\
DataSet     &           &           &         &           &           &         \\
\midrule
orig        &  0.991813 &         - &       - &  0.991729 &         - &       - \\
FMNIST      &  0.655473 &  0.977823 &  0.1485 &  0.487139 &   0.98104 &  0.1278 \\
EMNIST      &  0.805926 &  0.906156 &  0.3764 &  0.803764 &  0.903264 &  0.3754 \\
GrayCIFAR10 &  0.397742 &  0.998202 &  0.0027 &  0.100134 &   0.99995 &       0 \\
Noise       &  0.121303 &  0.999999 &       0 &  0.105445 &  0.999928 &       0 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &      FPR@95 &       MMC &     AUROC &  FPR@95 \\
DataSet     &           &           &             &           &           &         \\
\midrule
orig        &  0.9

In [4]:
device = torch.device('cuda:3')
dataset = 'FMNIST'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCAFMNIST_lam0.0_n100_lr0.001_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_MNIST_lr0.001_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_FMNIST_n100_data_used60000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

0.9424
0.9428


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.1218
0.1218
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &    FPR@95 &       MMC &     AUROC &    FPR@95 \\
DataSet     &           &           &           &           &           &           \\
\midrule
orig        &  0.971288 &         - &         - &  0.970874 &         - &         - \\
MNIST       &  0.778298 &   0.91646 &    0.2989 &  0.778298 &  0.915989 &    0.2989 \\
EMNIST      &  0.701693 &   0.93547 &    0.1973 &  0.701692 &  0.934992 &    0.1973 \\
GrayCIFAR10 &  0.861510 &  0.818207 &    0.5213 &  0.861342 &  0.817707 &    0.5213 \\
Noise       &  0.659809 &  0.940255 &  0.191953 &  0.652225 &  0.940252 &  0.193281 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &      AUROC &      FPR@95 &       MMC &      AUROC &      FPR@95 \\
DataSet     &           &            &             &           &            &   

In [5]:
device = torch.device('cuda:3')
dataset = 'SVHN'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCASVHN_lam0.0_n100_lr0.001_lrgmm1e-06_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_SVHN_lr0.01_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_SVHN_n100_data_used50000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

Using downloaded and verified file: ../data/train_32x32.mat
Using downloaded and verified file: ../data/train_32x32.mat
Using downloaded and verified file: ../data/test_32x32.mat
0.9595113706207744
Using downloaded and verified file: ../data/test_32x32.mat
0.9595497848801475
Using downloaded and verified file: ../data/test_32x32.mat


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.9690765212046711
Using downloaded and verified file: ../data/test_32x32.mat
0.9691917639827904
Using downloaded and verified file: ../data/test_32x32.mat
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &    FPR@95 &       MMC &     AUROC &    FPR@95 \\
DataSet   &           &           &           &           &           &           \\
\midrule
orig      &  0.975878 &         - &         - &  0.975859 &         - &         - \\
CIFAR10   &  0.726107 &    0.9309 &    0.2084 &  0.572675 &  0.949249 &    0.1526 \\
CIFAR100  &  0.736866 &  0.924342 &    0.2259 &  0.541490 &  0.948574 &    0.1553 \\
LSUN\_CR   &  0.728132 &  0.924333 &  0.216667 &  0.161476 &     0.992 &      0.02 \\
Imagenet- &  0.735721 &  0.927048 &    0.2261 &  0.212153 &  0.988216 &     0.038 \\
Noise     &  0.685477 &  0.946663 &   0.14751 &  0.682895 &  0.946834 &  0.145953 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \m

In [6]:
device = torch.device('cuda:3')
dataset = 'CIFAR10'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

file_joint = 'gmm__PCACIFAR10_lam0.0_n100_lr0.001_lrgmm1e-06_augm_flagTrue_train_typeCEDA_GMMgrad_vars mu var.pth'
file_base = 'base_CIFAR10_lr0.01_augm_flagTrue_train_typeCEDA.pth'
file_gmm = 'gmm_CIFAR10_n100_data_used50000_augm_flagTrue_alg_scikit_PCA.pth'

df_sep, df_joint = evaluate(model_params, device, file_joint, file_base, file_gmm)
print(df_joint.to_latex())
print(df_sep.to_latex())

Using downloaded and verified file: ../data/test_32x32.mat
0.8881
0.8886


/home/alexm/project/notebooks/gmm-robust/utils/models.py:306: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.loglam = nn.Parameter(torch.tensor(loglam, dtype=torch.float))


0.9303
0.9309
\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &    FPR@95 &       MMC &     AUROC &    FPR@95 \\
DataSet   &           &           &           &           &           &           \\
\midrule
orig      &  0.931302 &         - &         - &  0.930700 &         - &         - \\
SVHN      &  0.756881 &  0.845316 &    0.1989 &  0.756884 &  0.844335 &    0.1989 \\
CIFAR100  &  0.747215 &  0.830066 &    0.2409 &  0.744823 &  0.829108 &    0.2408 \\
LSUN\_CR   &  0.695819 &  0.860633 &  0.176667 &  0.682758 &    0.8652 &  0.166667 \\
Imagenet- &  0.734040 &  0.837583 &    0.2269 &  0.674854 &  0.851392 &    0.2077 \\
Noise     &  0.762270 &  0.837077 &  0.215625 &  0.762318 &  0.834172 &  0.217734 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrllrll}
\toprule
{} & \multicolumn{3}{l}{Base} & \multicolumn{3}{l}{GMM} \\
{} &       MMC &     AUROC &    FPR@95 &       MMC &     AUROC &    FPR@95 \\
DataSet   & 